Objectif : 

Recherche des doublons, explorer les lignes budgets liées et produire un csv lié (simple)

Outil : 
Echantillon de 20k fichiers 

In [1]:
import pandas as pd 

df_doc = pd.read_csv('../data/explo/doc_doublon_1')
df_budget = pd.read_parquet('../data/bloc_budget_20.parquet')

/tmp/ipykernel_4394/2398582516.py:3: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_doc = pd.read_csv('../data/explo/doc_doublon_1')


In [3]:
df_doc = df_doc.drop(columns='Unnamed: 0')
doc_doublon = df_doc[df_doc.duplicated(subset=['Siret', 'Exer', 'NatDec', 'NumDec','CodTypBud'], keep = False)]
doc_doublon[['Id_Fichier', 'Siret', 'NatDec','NumDec','CodTypBud', 'Date']].sort_values(['Siret', 'NumDec']).head(5)

,Id_Fichier,Siret,NatDec,NumDec,CodTypBud,Date
5104,790376,20000480200027,2,1.0,A,2021-11-03T08:23:51.768+01:00
10389,791034,20000480200027,2,1.0,A,2021-11-08T16:01:39.443+01:00
6262,681458,20000797900012,1,NaN,P,2021-04-01T12:43:44.440+02:00
19949,681423,20000797900012,1,NaN,P,2021-04-01T12:43:44.440+02:00
6154,804914,20002715900017,1,NaN,P,2021-12-22T16:15:22


On va séparer en deux catégories : ceux qui ont la même date et ceux qui ont des dates différentes

In [5]:
doublon_meme_date = doc_doublon[doc_doublon.duplicated(subset= ['Date'], keep = False)].sort_values(['Siret', 'Date'])
doublon_diff_date = doc_doublon[~doc_doublon.duplicated(subset= ['Date'], keep = False)].sort_values(['Siret', 'Date'])

print('pure doublon',len(doublon_meme_date))
print('modif posteriori', len(doublon_diff_date))

pure doublon 141
modif posteriori 201


In [20]:
doublon_diff_date['Plus_recent'] = (doublon_diff_date['Date'] == doublon_diff_date.groupby('Siret')['Date'].transform('max'))


In [15]:
colonnes_doc_budgetaire_meme_date = [
 'Id_Fichier',
 'Nomenclature',
 'Exer',
 'Siret',
 'Siren',
 'CodColl',
 'LibelleColl',
 'DteStr',
 'Date',
 'DteDec',
 'DteDecEx',
 'NumDec',
 'IdPost',
 'LibellePoste',
 'LibelleEtabPal',
 'IdEtabPal',
 'LibelleEtab',
 'IdEtab',
 'NatDec',
 'NatVote', 	
 'OpeEquip', 
 'CodInseeColl',	
 'VoteFormelChap', 	
 'TypProv', 	
 'BudgPrec',
 'RefProv',	
 'ReprRes', 	
 'NatFonc', 	
 'PresentationSimplifiee', 	
 'DepFoncN2',	
 'RecFoncN2' ,	
 'DepInvN2' ,	
 'RecInvN2' ,	
 'CodTypBud',
 'CodBud',
 'ProjetBudget',  	
 'Affect',
 'SpecifBudget',
 'FinJur',
 'Md5',
 'Sha1'	 	
 ]

colonnes_doc_budgetaire_diff_date = [
 'Id_Fichier',
 'Plus_recent',
 'Nomenclature',
 'Exer',
 'Siret',
 'Siren',
 'CodColl',
 'LibelleColl',
 'DteStr',
 'Date',
 'DteDec',
 'DteDecEx',
 'NumDec',
 'IdPost',
 'LibellePoste',
 'LibelleEtabPal',
 'IdEtabPal',
 'LibelleEtab',
 'IdEtab',
 'NatDec',
 'NatVote', 	
 'OpeEquip', 
 'CodInseeColl',	
 'VoteFormelChap', 	
 'TypProv', 	
 'BudgPrec',
 'RefProv',	
 'ReprRes', 	
 'NatFonc', 	
 'PresentationSimplifiee', 	
 'DepFoncN2',	
 'RecFoncN2' ,	
 'DepInvN2' ,	
 'RecInvN2' ,	
 'CodTypBud',
 'CodBud',
 'ProjetBudget',  	
 'Affect',
 'SpecifBudget',
 'FinJur',
 'Md5',
 'Sha1'	 	
 ]


In [19]:
doublon_meme_date = doublon_meme_date[colonnes_doc_budgetaire_meme_date]
doublon_diff_date = doublon_diff_date[colonnes_doc_budgetaire_diff_date]

In [29]:
doublon_meme_date.to_csv('../data/explo/doc_budgetaire_doublon_memes_dates')
doublon_diff_date.to_csv('../data/explo/doc_budgetaire_doublon_diff_dates')
doc_doublon.to_csv('../data/explo/doc_budgetaire_doublon')

--------
Explo des lignes budgets mêmes dates 

-------------

Prenons l'exemple de 681458 et 681423

In [23]:
budget_691458 = df_budget[df_budget['Id_Fichier'] == '681458']
budget_681423 = df_budget[df_budget['Id_Fichier'] == '681423']

budget_1er_doublon = df_budget[df_budget['Id_Fichier'].isin(['681458', '681423'])]

print('taille premier :', budget_691458.shape[0])
print('taille seconnd :', budget_681423.shape[0])


taille premier : 103
taille seconnd : 103


Si les lignes sont les mêmes, on passera de 206 lignes à 103. 

On va d'avoir retirer l'Id_Fichier qui est évidemment différent (et qui devrait être la seule diff)

In [24]:
col_autres_que_id = budget_1er_doublon.columns.difference(['Id_Fichier'])

budget_test_unique = budget_1er_doublon.drop_duplicates(subset=col_autres_que_id, keep = 'first')
print(budget_test_unique.shape[0], '\n')
print(budget_test_unique['Id_Fichier'].value_counts())

103 

Id_Fichier
681458    103
Name: count, dtype: int64


Maintenant, il suffit de généraliser ça 

In [25]:
liste_id_doublon_meme = doublon_meme_date['Id_Fichier'].to_list()

df_budget_doublon_meme = df_budget[df_budget['Id_Fichier'].isin(liste_id_doublon_meme)]

print('doublon pure :',df_budget_doublon_meme.shape[0], 'lignes budgets \n')

doublon pure : 0 lignes budgets 



Certains sont en triples donc on peut pas juste diviser par deux

In [26]:
df_budget_unique_meme_f = df_budget_doublon_meme.drop_duplicates(subset=col_autres_que_id, keep = False)

print('Il y a ', df_budget_unique_meme_f.shape[0], 'lignes budgets unique \
dans les données budgets des documents doublons envoyés au même moment')


Il y a  0 lignes budgets unique dans les données budgets des documents doublons envoyés au même moment


--------------
Maintenant, recherche chez ceux qui ont une diff 

-----------


Prenons les id_fichier 790376 et 791034

In [27]:
budget_790376 = df_budget[df_budget['Id_Fichier'] == '790376']
budget_791034 = df_budget[df_budget['Id_Fichier'] == '791034']
budget_test_diff = df_budget[df_budget['Id_Fichier'].isin(['791034', '790376'])]

print(budget_790376.shape[0])
print(budget_791034.shape[0])

131
131


In [28]:
budget_dif_unique = budget_test_diff.drop_duplicates(subset=col_autres_que_id, keep = 'first')
print(budget_dif_unique.shape[0], '\n')
print(budget_dif_unique['Id_Fichier'].value_counts())

172 

Id_Fichier
790376    131
791034     41
Name: count, dtype: int64
